In [1]:
# Questions

In [2]:
import requests
import json
from elasticsearch import Elasticsearch

In [3]:
from dotenv import load_dotenv
load_dotenv()

import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [4]:
def llm_response(prompt):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gemma2-9b-it",
        temperature=0.2,
    )
    return response.choices[0].message.content

In [5]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [6]:
documents = []

for course in documents_raw:
    course_name = course["course"]
    
    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

In [7]:
es_client = Elasticsearch("http://elasticsearch:9200")

In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [9]:
index_name ="homework_1_course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework_1_course-questions'})

In [10]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [26]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                #Comment the next lines to answer Q3
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response =  es_client.search(index=index_name, body=search_query)
    result_docs=[]
    for hit in response["hits"]["hits"]:
        result_docs.append(hit)
    
    return result_docs

# Q3

In [27]:
query = "How do execute a command on a Kubernetes pod?"

In [34]:
scores_search_results = elastic_search(query)
scores_search_results
scores =[doc["_score"] for doc in scores_search_results]
scores

[44.50556, 35.433445, 33.70974]

# Q4

In [14]:
query = "How do copy a file to a Docker container?"

In [15]:
questions_search_results = elastic_search(query)
questions = [doc["question"] for doc in questions_search_results]

In [16]:
questions

['How do I debug a docker container?',
 'How do I copy files from my local machine to docker container?',
 'How do I copy files from a different folder into docker container’s working directory?']

# Q5

In [17]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    context = ""
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt 

In [18]:
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)
print(len(prompt))

1446


# Q6

In [19]:
# !pip install tiktoken

In [20]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [21]:
# encoding.encode(prompt) returns int, each int is an id that represents a specific token, like a word or sub-word unit
prompt_tokens = len(encoding.encode(prompt))
prompt_tokens

320

# BONUS

## B1

In [22]:
def llm_response(prompt):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gemma2-9b-it",
        temperature=0.2,
    )
    return response.choices[0].message.content

In [23]:
answer = llm_response(prompt)
print(query + "\n\n" + answer)

How do copy a file to a Docker container?

You can copy a file to a Docker container using the `docker cp` command. 

The basic syntax is:

`docker cp /path/to/local/file_or_directory container_id:/path/in/container` 



## B2

In [24]:
answer_tokens = len(encoding.encode(answer))
answer_tokens

41

In [25]:
#if we used the prices/token for gpt4o:
input_price = 0.005 / 1000
output_price = 0.015 / 1000
#then

final_input_price = prompt_tokens * input_price
final_output_price = answer_tokens * output_price

final_price = final_input_price + final_output_price
print(f"the total price is ${final_price}")

the total price is $0.002215
